> <div class = 'markdown-google-sans'><h1>Deep Learning Backpropagation Algorithm for Regression</h1>
> <h2>1. Forward Pass</h2>
> <ul>
    <li><strong>Input Layer:</strong> Input features are fed into the network.</li>
    <li><strong>Hidden Layers:</strong> Each neuron computes a weighted sum of the inputs plus a bias term. For linear activation, the output is simply the weighted sum itself:
        <p><code>A = W &middot; X + b</code></p>
    </li>
    <li><strong>Output Layer:</strong> Computes a weighted sum of the last hidden layer’s activations plus a bias term to produce the final prediction.</li>
> </ul><h2>2. Calculate Loss</h2><ul>
    <li><strong>Loss Function (Mean Squared Error):</strong> Measures the average squared difference between predicted and actual target values:
        <p><code>L = &frac12; &sum; (y<sub>i</sub> - &yhat;<sub>i</sub>)<sup>2</sup></code></p>
        where <code>y<sub>i</sub></code> is the true value, <code>&yhat;<sub>i</sub></code> is the predicted value, and <code>n</code> is the number of samples.
    </li>
> </ul><h2>3. Backward Pass (Backpropagation)</h2><ul>
    <li><strong>Compute Gradients:</strong> Calculate the gradient of the loss with respect to each weight and bias using the chain rule.</li>
    <li><strong>Update Weights and Biases:</strong> Use an optimization algorithm (like Gradient Descent) to update weights and biases to minimize the loss function:
        <p><code>W = W - &eta; &times; (∂L / ∂W)</code></p>
        <p><code>b = b - &eta; &times; (∂L / ∂b)</code></p>
        where <code>&eta;</code> is the learning rate.
    </li>
> </ul><h2>4. Repeat</h2><ul>
    <li>Iterate through the forward pass, loss calculation, backpropagation, and weight update steps for multiple epochs until the model performs satisfactorily.</li>
</ul>
</div>

In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [19]:
# Dataset
df = pd.DataFrame([[8,8,4], [7, 9, 5], [6, 10, 6], [5, 12, 7]], columns = ['cgpa', 'resume_score', 'package'])
df

,cgpa,resume_score,package
0,8,8,4
1,7,9,5
2,6,10,6
3,5,12,7


> <div class = "markdown-google-sans"><h1>Function 1 - Parameter Initialization</h1>

In [20]:
def parameter_initializaton(ann_dim):
    # ann_dim is the ann architecture - [2,2,1]
    '''
    parameters = {
        w1 = [[w11, w12], [w21, w22]], -> Layer 1
        w2 = [[w11], [w21]]            -> Layer 2
        b1 = [[b1, b2]]                -> Layer 1
        b2 = [[b1]]                    -> Layer 2
    }
    '''
    parameters = {}
    length = len(ann_dim)

    for i in range(1, length):
        parameters['w' + str(i)] = np.ones((ann_dim[i-1], ann_dim[i])) * 0.1
        parameters['b' + str(i)] = np.zeros((1, ann_dim[i]))

    return parameters

In [21]:
parameters = parameter_initializaton([2,2,1])
parameters

{'w1': array([[0.1, 0.1],
        [0.1, 0.1]]),
 'b1': array([[0., 0.]]),
 'w2': array([[0.1],
        [0.1]]),
 'b2': array([[0.]])}

> <div class = "markdown-google-sans"><h1>Function 2 - Forward Propagation</h1>

In [22]:
def forward_propagation(X, parameters):
    layers = len(parameters) // 2 # no. of layers

    A = X
    # We have to calculate the y_hat layer by layer
    for i in range(1, layers + 1):
        # Fetching previous Result
        A_prev = A

        # Fetching the parameters of current layers
        w = parameters['w' + str(i)]
        b = parameters['b' + str(i)]

        # Dot product
        A = np.dot(A_prev, w) + b

    return A, A_prev

In [23]:
A, A1 = forward_propagation(df.iloc[[0], :-1], parameters)
print(A, A1)

[[0.32]] [[1.6 1.6]]


> <div class = "markdown-google-sans"><h1>Function 3 - Updating Weights & Biases</h1>

In [24]:
def update_parameters(y, y_hat, A1, X, parameters):
    # For 2nd layer
    parameters['w2'][0][0] += (0.001 * 2 * (y - y_hat)*A1[0][0])
    parameters['w2'][1][0] += (0.001 * 2 * (y - y_hat)*A1[0][1])
    parameters['b2'][0][0] += (0.001 * 2 * (y - y_hat))

    # For 1st layer
    parameters['w1'][0][0] =+ (0.01 * 2 * (y - y_hat)*parameters['w2'][0][0]*X[0][0])
    parameters['w1'][0][1] =+ (0.01 * 2 * (y - y_hat)*parameters['w2'][0][0]*X[0][1])
    parameters['b1'][0][0] =+ (0.01 * 2 * (y - y_hat)*parameters['w2'][0][0])
    parameters['w1'][1][0] =+ (0.01 * 2 * (y - y_hat)*parameters['w2'][1][0]*X[0][0])
    parameters['w1'][1][1] =+ (0.01 * 2 * (y - y_hat)*parameters['w2'][1][0]*X[0][1])
    parameters['b1'][0][1] =+ (0.01 * 2 * (y - y_hat)*parameters['w2'][1][0])

In [34]:
update_parameters(df['package'][0], A, A1, np.array(df.iloc[:1, :-1]), parameters)

In [35]:
parameters

{'w1': array([[0.07968113, 0.07968113],
        [0.07968113, 0.07968113]]),
 'b1': array([[0.00996014, 0.00996014]]),
 'w2': array([[0.135328],
        [0.135328]]),
 'b2': array([[0.02208]])}

> <div class = "markdown-google-sans"><h1>Looping on the complete dataset for 10 epochs</h1>

In [40]:
# Initilizing parameters
parameters = parameter_initializaton([2,2,1])

for i in range(10):
    loss = []
    for j in range(df.shape[0]):
        # Forward Propogation
        y_hat, A1 = forward_propagation(df.iloc[[j], :-1], parameters)

        # Loss Calculation
        curr_loss = (df['package'][j] - y_hat)**2
        loss.append(curr_loss)

        # Updsting w and b
        update_parameters(df['package'][j], y_hat, A1, np.array(df.iloc[j:j+1,:-1]), parameters)

    print(np.array(loss).mean())

27.088241783863353
21.130090574227584
14.94196219445731
11.915901602185107
14.320929059557596
21.615150010534634
18.534456274833314
15.434241042829964
16.930507784354287
18.147401123895506


In [41]:
parameters # Updated parameters after 10 epochs

{'w1': array([[0.19399769, 0.46559446],
        [0.25425222, 0.61020532]]),
 'b1': array([[0.03879954, 0.05085044]]),
 'w2': array([[0.34599244],
        [0.45345561]]),
 'b2': array([[0.26923429]])}

<hr>

> <div class = "markdown-google-sans"><h1>Backpropogation using Tensorflow - Keras</h1>

In [42]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import InputLayer, Dense

In [43]:
model = Sequential()

model.add(Dense(2, activation = 'linear', input_dim = 2))
model.add(Dense(1, activation = 'linear'))

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 2)                   │               6 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │               3 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9 (36.00 B)

 Trainable params: 9 (36.00 B)

 Non-trainable params: 0 (0.00 B)

In [44]:
model.get_weights()

[array([[ 0.7146908 , -0.17461371],
        [ 0.80750644, -0.91531837]], dtype=float32),
 array([0., 0.], dtype=float32),
 array([[-0.39624643],
        [-1.3753291 ]], dtype=float32),
 array([0.], dtype=float32)]

In [45]:
# Define weights and biases manually
weights_for_layer1 = np.array([[0.1, 0.1], [0.1, 0.1]])
biases_for_layer1 = np.array([0.1, 0.1])
weights_for_layer2 = np.array([[0.1], [0.1]])
biases_for_layer2 = np.array([0.1])

# Combine them into the correct format
parameters_format = [weights_for_layer1, biases_for_layer1, weights_for_layer2, biases_for_layer2]

# Set the weights
model.set_weights(parameters_format)

In [46]:
model.get_weights()

[array([[0.1, 0.1],
        [0.1, 0.1]], dtype=float32),
 array([0.1, 0.1], dtype=float32),
 array([[0.1],
        [0.1]], dtype=float32),
 array([0.1], dtype=float32)]

In [47]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01), loss='mse', metrics=['accuracy'])

In [48]:
model.fit(df.iloc[:, :-1].values, df['package'].values, epochs = 10)

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.0000e+00 - loss: 26.7881
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.0000e+00 - loss: 25.9582
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.0000e+00 - loss: 25.0732
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.0000e+00 - loss: 24.1360
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.0000e+00 - loss: 23.1497
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.0000e+00 - loss: 22.1179
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.0000e+00 - loss: 21.0444
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.0000e+00 - loss: 19.9336
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.0000e+00 - loss: 18.7904
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.0000e+00 - loss: 17.6202


> <div class = "markdown-google-sans">Since the loss values of our custom backpropagation model and the Keras backpropagation model are identical, this verifies that our model implementation is correct."</div>